In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\mohda\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY = "pcsk_5ftJtk_RPGwskkYfaFGx21SBduyqwS44EGQ2stdCX3rJRdhB1s32dos2WfAJMH6hRDhq3B"

PINECONE_API_ENV = "us-east-1"

In [3]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [4]:
import pdfplumber

def load_pdf(pdf_path):
    """Extract text from a PDF using pdfplumber."""
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Extract text
extracted_data = load_pdf(r"C:\Users\mohda\OneDrive\Desktop\MEDICAL-CHATBOT\data\Medical_book.pdf")
print(extracted_data[:500])
 


GALE
The
ENCYCLOPEDIA
M
EDICINE
of
SECOND EDITION
G ALE
The
ENCYCLOPEDIA
M
EDICINE
of
SECOND EDITION
V O L U M E
1
A-B
JACQUELINE L. LONGE, EDITOR
DEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR
The GALE Since this page cannot legibly accommodate all copyright notices,the
acknowledgments constitute an extension of the copyright notice.
ENCYCLOPEDIA
While every effort has been made to ensure the reliability of the infor-
mation presented in this publication,the Gale Group neither guarantees
of MEDICINE


In [5]:
# extracted_data

In [6]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [8]:

# Split extracted text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
text_chunks = text_splitter.split_text(extracted_data)

print(f"Extracted {len(text_chunks)} text chunks from PDF.")


Extracted 5720 text chunks from PDF.


In [9]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [10]:

embeddings = download_hugging_face_embeddings()


In [11]:

from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# Load the embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
from langchain.embeddings import HuggingFaceEmbeddings

# Load the embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
from langchain.embeddings import HuggingFaceEmbeddings

# 🔹 Load the Sentence Transformer model for embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 🔹 Convert a sample text into an embedding vector
text = "How does the heart function?"
query_embedding = embedding_model.embed_query(text)

print("Generated Embedding Vector:", query_embedding[:10])  # Print first 10 values


Generated Embedding Vector: [-0.004146425519138575, 0.05755056068301201, -0.001845396007411182, 0.014783775433897972, -0.04701027646660805, 0.01919114775955677, 0.003776268567889929, -0.07476205378770828, 0.04135068878531456, -0.024365302175283432]


In [12]:
embeddings = download_hugging_face_embeddings()


In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
from pinecone import Pinecone

# 🔹 Step 1: Initialize Pinecone Client
pc = Pinecone(api_key="pcsk_5ftJtk_RPGwskkYfaFGx21SBduyqwS44EGQ2stdCX3rJRdhB1s32dos2WfAJMH6hRDhq3B")  # Replace with your API key

# 🔹 Step 2: Define Index Name and Region
index_name = "medicalchatbot"
region = "us-east-1"  # Change this to match your Pinecone region


# Connect to the existing index
index = pc.Index(index_name)
print(f"Connected to index: {index_name}")


Connected to index: medicalchatbot


In [16]:
import fitz  # PyMuPDF for PDF text extraction
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pinecone import Pinecone, ServerlessSpec
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as PineconeVectorStore

# 🔹 Step 1: Extract Text from Uploaded PDF
def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# Path to the uploaded PDF
pdf_path = r"C:\Users\mohda\OneDrive\Desktop\MEDICAL-CHATBOT\data\Medical_book.pdf"  # Replace with the actual path

# Extract text from PDF
extracted_text = extract_text_from_pdf(pdf_path)

# 🔹 Step 2: Split Extracted Text into Smaller Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_chunks = text_splitter.split_text(extracted_text)

print(f"✅ Extracted {len(text_chunks)} text chunks from PDF.")

# 🔹 Step 3: Initialize Pinecone Client
pc = Pinecone(api_key="pcsk_5ftJtk_RPGwskkYfaFGx21SBduyqwS44EGQ2stdCX3rJRdhB1s32dos2WfAJMH6hRDhq3B")  # Replace with your actual API Key

# Define Pinecone index details
index_name = "medicalchatbot"
expected_dim = 384  # Ensure embeddings match this dimension



# Connect to the existing index
index = pc.Index(index_name)
print(f"Connected to index: {index_name}")

# 🔹 Step 4: Convert Text Chunks into Vector Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedding_model.embed_documents(text_chunks)

# 🔹 Step 5: Format Data for Pinecone Upsert
vectors = [
    {"id": f"doc_{i}", "values": embeddings[i], "metadata": {"text": text_chunks[i]}}
    for i in range(len(text_chunks))
]

# 🔹 Step 6: Store Vectors in Pinecone
batch_size = 100  # To prevent exceeding Pinecone's request limits
for i in range(0, len(vectors), batch_size):
    index.upsert(vectors=vectors[i : i + batch_size])
    print(f"Upserted {i + len(vectors[i : i + batch_size])} vectors so far.")

print("✅ PDF text chunks successfully converted to vectors and stored in Pinecone!")


✅ Extracted 3392 text chunks from PDF.
Connected to index: medicalchatbot
Upserted 100 vectors so far.
Upserted 200 vectors so far.
Upserted 300 vectors so far.
Upserted 400 vectors so far.
Upserted 500 vectors so far.
Upserted 600 vectors so far.
Upserted 700 vectors so far.
Upserted 800 vectors so far.
Upserted 900 vectors so far.
Upserted 1000 vectors so far.
Upserted 1100 vectors so far.
Upserted 1200 vectors so far.
Upserted 1300 vectors so far.
Upserted 1400 vectors so far.
Upserted 1500 vectors so far.
Upserted 1600 vectors so far.
Upserted 1700 vectors so far.
Upserted 1800 vectors so far.
Upserted 1900 vectors so far.
Upserted 2000 vectors so far.
Upserted 2100 vectors so far.
Upserted 2200 vectors so far.
Upserted 2300 vectors so far.
Upserted 2400 vectors so far.
Upserted 2500 vectors so far.
Upserted 2600 vectors so far.
Upserted 2700 vectors so far.
Upserted 2800 vectors so far.
Upserted 2900 vectors so far.
Upserted 3000 vectors so far.
Upserted 3100 vectors so far.
Upser

In [17]:
from langchain.vectorstores import Pinecone as PineconeVectorStore
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone

# 🔹 Initialize Pinecone Client
pc = Pinecone(api_key="pcsk_5ftJtk_RPGwskkYfaFGx21SBduyqwS44EGQ2stdCX3rJRdhB1s32dos2WfAJMH6hRDhq3B")  # Replace with your API Key





# 🔹 Step 2: Define Index Name
index_name = "medicalchatbot"

# 🔹 Step 3: Get the Host of the Index
index_info = pc.describe_index(index_name)
host_url = "https://medicalchatbot-kvk4kva.svc.aped-4627-b74a.pinecone.io"  # Extract host URL

# 🔹 Step 4: Connect to Pinecone Index with Host
index = pc.Index(index_name, host=host_url)

print(f"✅ Connected to Pinecone index: {index_name}")

index_name = "medicalchatbot"


# 🔹 Initialize Embedding Model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 🔹 Connect to Existing Pinecone Index

# 🔹 Define Query
query = "what are headaches?"



query_result = index.query(
    vector=query_embedding,
    top_k=5,  # More results
    include_metadata=True
)

print("\n🔍 Query Results:")
for match in query_result["matches"]:
    score = match["score"]
    text = match["metadata"].get("text", "⚠ No matching text found")
    print(f"🔹 Confidence Score: {score:.4f}")
    print(f"📄 Retrieved Text: {text}\n")





✅ Connected to Pinecone index: medicalchatbot

🔍 Query Results:
🔹 Confidence Score: 0.5543
📄 Retrieved Text: ⚠ No matching text found

🔹 Confidence Score: 0.5457
📄 Retrieved Text: ⚠ No matching text found

🔹 Confidence Score: 0.5374
📄 Retrieved Text: heart are attached to the chest and extremities. The infor-
mation is displayed on a monitor screen or a paper tape
in the form of waves. Reduced blood and oxygen supply
to the heart shows as a change in the shape of the waves.
• Noninvasive vascular tests. These involve measuring
blood pressure in various parts of the body and compar-
ing the results from each location. When there is a
KEY TERMS
Arterial fibrillation—An arrhythmia; chaotic quiv-
ering of the arteries.

🔹 Confidence Score: 0.5083
📄 Retrieved Text: Prognosis
The prognosis for aortic valve stenosis depends on
the severity of the disease. With surgical repair, the dis-
KEY TERMS
Aorta—The largest artery in the body, which
moves blood from the left ventricle to the rest of
the

In [18]:
# Describe your Pinecone index
index_stats = index.describe_index_stats()
print("Index Stats:", index_stats)


Index Stats: {'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 11096},
                'medical_book_text': {'vector_count': 2}},
 'total_vector_count': 11098}


In [19]:
query_result = index.query(
    vector=[0] * 384,  # Dummy query vector
    top_k=5,
    include_metadata=True
)

# Print stored text chunks
print("\n🔍 Stored Data in Pinecone:")
for match in query_result["matches"]:
    print(f"📄 Text: {match['metadata'].get('text', 'No text found')}")



🔍 Stored Data in Pinecone:
📄 Text: No text found
📄 Text: able deformity. However, with careful attention paid to
the development of dangerous complications (nerve
compression, hydrocephalus), most people are in good
health, and can live a normal lifespan.
Prevention
The only form of prevention is through genetic
counseling, which could help parents assess their risk of
having a child with achondroplasia.
Resources
BOOKS
Hall, Bryan D. “Achondroplasia.” In Nelson Textbook of Pedi-
atrics, ed. Richard E. Behrman. Philadelphia: W. B. Saun-
ders Co., 1996.
Horton, W. A., and J. T. Hecht. “The Chondrodysplasias.” In
Connective Tissue and Its Heritable Disorders, ed. R. M.
Royce and B. Steinmann. Somerset, NJ: Wiley-Liss, 1993.
Krane, Stephen M., and Alan L. Schiller. “Achondroplasia.” In
Harrison’s Principles of Internal Medicine, ed. Anthony
S. Fauci, et al. New York: McGraw-Hill, 1997.
Whyte, Michael, P. “Achondroplasia.” In Cecil Textbook of
Medicine, ed. J. Claude Bennett and Fred Plum

In [20]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# Load the same embedding model used for storage
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Convert query into vector
query_text = "What are fractures?"
query_embedding = embedding_model.embed_query(query_text)


In [21]:
query_result = index.query(
    vector=query_embedding, 
    top_k=5, 
    include_metadata=True
)

print("\n🔍 Query Results:")
for match in query_result["matches"]:
    score = match["score"]
    text = match["metadata"].get("text", "⚠ No matching text found")
    print(f"🔹 Confidence Score: {score:.4f}")
    print(f"📄 Retrieved Text: {text}\n")



🔍 Query Results:
🔹 Confidence Score: 0.5550
📄 Retrieved Text: ⚠ No matching text found

🔹 Confidence Score: 0.5467
📄 Retrieved Text: is the development of infection at the site of implantation.
Normal results
Success in healing a fracture nonunion using bone
growth stimulation depends on the type, location, and
severity of the fracture and the age and general health of
the patient.
Resources
PERIODICALS
Mayo Clinic. “Fractures - Treatment Methods are Tailored to
the Break.” Mayo Clinic Health Letter 14, no. 4
(Apr.1996):1-3.
Tish Davidson
Bone infection see Osteomyelitis
Bone marrow aspiration 
and biopsy
Definition

🔹 Confidence Score: 0.5281
📄 Retrieved Text: wear and tear, age and less often from inflammation.
Osteogenesis imperfecta—Also called brittle bones,
this is a condition present at birth in which bones
are abnormally fragile, brittle and break easily.
Osteomalacia—A disease in which bones gradual-
ly soften and bend.
Osteomyelitis—An infection of the bone marrow
and the bo

In [22]:
query_text = "What is the cure for blisters?"
query_embedding = embedding_model.embed_query(query_text)

In [23]:
query_result = index.query(
    vector=query_embedding, 
    top_k=5, 
    include_metadata=True
)

print("\n🔍 Query Results:")
for match in query_result["matches"]:
    score = match["score"]
    text = match["metadata"].get("text", "⚠ No matching text found")
    print(f"🔹 Confidence Score: {score:.4f}")
    print(f"📄 Retrieved Text: {text}\n")



🔍 Query Results:
🔹 Confidence Score: 0.6274
📄 Retrieved Text: quickly blisters and deteriorates into open sores that can
harbor life-threatening infection.
GALE ENCYCLOPEDIA OF MEDICINE 2
456
Bedsores

🔹 Confidence Score: 0.5638
📄 Retrieved Text: blister may have to be drained, and a heating pad may
help swollen, tender lymph glands. Acetaminophen
(Tylenol) may relieve pain, aches, and fever over 101°F
(38.3°C).
Prognosis
In most cases, prompt antibiotic treatment in
patients with AIDS cured the infection caused by either
variety of the bacteria, and patients may resume normal
life. Early diagnosis is crucial to a cure.
Prevention
Studies suggest that antibiotics may prevent the dis-

🔹 Confidence Score: 0.5443
📄 Retrieved Text: applied to the burn since it prevents heat from escaping
and drives the burning process deeper into the skin.
If the burn is minor, it may be cleaned gently with
soap and water. Blisters should not be broken. If the skin
of the burned area is unbroken and it i

In [24]:
query_text = "what is HIV?"
query_embedding = embedding_model.embed_query(query_text)

In [25]:
query_result = index.query(
    vector=query_embedding, 
    top_k=5, 
    include_metadata=True
)

print("\n🔍 Query Results:")
for match in query_result["matches"]:
    score = match["score"]
    text = match["metadata"].get("text", "⚠ No matching text found")
    print(f"🔹 Confidence Score: {score:.4f}")
    print(f"📄 Retrieved Text: {text}\n")


🔍 Query Results:
🔹 Confidence Score: 0.7456
📄 Retrieved Text: ⚠ No matching text found

🔹 Confidence Score: 0.6977
📄 Retrieved Text: ⚠ No matching text found

🔹 Confidence Score: 0.6851
📄 Retrieved Text: ⚠ No matching text found

🔹 Confidence Score: 0.6682
📄 Retrieved Text: protein molecules suspended in a gel by running an
electric current through the gel.
Enzyme-linked immunosorbent assay (ELISA)—A
diagnostic blood test used to screen patients for
AIDS or other viruses. The patient’s blood is mixed
with antigen attached to a plastic tube or bead sur-
face. A sample of the patient’s blood serum is
added, and excess proteins are removed. A second
antibody coupled to an enzyme is added, followed
by a chemical that will cause a color reaction that
can be measured by a special instrument.
Human immunodeficiency virus (HIV)—A transmis-
sible retrovirus that causes AIDS in humans. Two
forms of HIV are now recognized: HIV-1, which caus-
es most cases of AIDS in Europe, North and South
Ameri

In [26]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [27]:
from langchain.prompts import PromptTemplate


PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [28]:
from langchain.chains import RetrievalQA
from langchain.llms import CTransformers

llm = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGUF", model_type="llama")




Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 1000.31it/s]


ValidationError: 1 validation error for CTransformers
__root__
  No model file found in repo 'TheBloke/Llama-2-7B-Chat-GGUF' (type=value_error)

In [29]:
query_text = "explain about adhd in detail"
query_embedding = embedding_model.embed_query(query_text)

In [30]:
query_result = index.query(
    vector=query_embedding, 
    top_k=10, 
    include_metadata=True
)

print("\n🔍 Query Results:")
for match in query_result["matches"]:
    score = match["score"]
    text = match["metadata"].get("text", "⚠ No matching text found")
    print(f"🔹 Confidence Score: {score:.4f}")
    print(f"📄 Retrieved Text: {text}\n")


🔍 Query Results:
🔹 Confidence Score: 0.7710
📄 Retrieved Text: disorder (ADHD)
Definition
Attention-deficit/hyperactivity disorder (ADHD) is
a developmental disorder characterized by distractibility,
hyperactivity, impulsive behaviors, and the inability to
remain focused on tasks or activities.
Description
ADHD, also known as hyperkinetic disorder (HKD)
outside of the United States, is estimated to affect 3-9%
of children, and afflicts boys more often than girls.
Although difficult to assess in infancy and toddlerhood,

🔹 Confidence Score: 0.7261
📄 Retrieved Text: tionally balanced diet is important for normal develop-
ment in all children.
Psychologists and other mental health professionals
typically use the criteria listed in the Diagnostic and Sta-
tistical Manual of Mental Disorders, Fourth Edition
(DSM-IV) as a guideline for determining the presence of
ADHD. For a diagnosis of ADHD, DSM-IV requires the
presence of at least six of the following symptoms of
inattention, or six or mo